In [1]:
from tools import *
from load_data import *
from scraper import *

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('../model/5_2_2020_data.csv')

In [4]:
def format_date_race(date, race):
    year, month, day = format_to_date(date)
    race = format_to_race(race)
    return [year, month, day, race]

In [5]:
def write_to_txt(path, file_name, content):
    with open(os.path.join(path, file_name), 'w') as f:
        f.writelines(content)

In [7]:
info = [format_date_race(j[0],j[1]) for i,j \
        in data[['file_name','sheet_name']].drop_duplicates().iterrows()]
path = '../../extra_size_files/data/5_1_2020_info'

In [8]:
failed_info = load_hkjc_page_info(info, path)

Loaded page data: 2018/04/11, Race 1
All data wrote to files

Loaded page data: 2018/04/11, Race 2
All data wrote to files

Loaded page data: 2018/04/11, Race 3
All data wrote to files

Loaded page data: 2018/04/11, Race 4
All data wrote to files

Loaded page data: 2018/04/11, Race 5
All data wrote to files

Loaded page data: 2018/04/11, Race 6
All data wrote to files

Loaded page data: 2018/04/11, Race 7
All data wrote to files

Loaded page data: 2018/04/11, Race 8
All data wrote to files

Loaded page data: 2018/04/15, Race 1
All data wrote to files

Loaded page data: 2018/04/15, Race 10
All data wrote to files

Loaded page data: 2018/04/15, Race 11
All data wrote to files

Loaded page data: 2018/04/15, Race 2
All data wrote to files

Loaded page data: 2018/04/15, Race 3
All data wrote to files

Loaded page data: 2018/04/15, Race 4
All data wrote to files

Loaded page data: 2018/04/15, Race 5
All data wrote to files

Loaded page data: 2018/04/15, Race 6
All data wrote to files

Loaded

Loaded page data: 2018/05/30, Race 4
All data wrote to files

Loaded page data: 2018/05/30, Race 5
All data wrote to files

Loaded page data: 2018/05/30, Race 6
All data wrote to files

Loaded page data: 2018/05/30, Race 7
All data wrote to files

Loaded page data: 2018/05/30, Race 8
All data wrote to files

Loaded page data: 2018/06/03, Race 1
All data wrote to files

Loaded page data: 2018/06/03, Race 10
All data wrote to files

Loaded page data: 2018/06/03, Race 2
All data wrote to files

Loaded page data: 2018/06/03, Race 3
All data wrote to files

Loaded page data: 2018/06/03, Race 4
All data wrote to files

Loaded page data: 2018/06/03, Race 5
All data wrote to files

Loaded page data: 2018/06/03, Race 6
All data wrote to files

Loaded page data: 2018/06/03, Race 7
All data wrote to files

Loaded page data: 2018/06/03, Race 8
All data wrote to files

Loaded page data: 2018/06/03, Race 9
All data wrote to files

Loaded page data: 2018/06/06, Race 1
All data wrote to files

Loaded 

Loaded page data: 2018/09/05, Race 5
All data wrote to files

Loaded page data: 2018/09/05, Race 6
All data wrote to files

Loaded page data: 2018/09/05, Race 7
All data wrote to files

Loaded page data: 2018/09/05, Race 8
All data wrote to files

Loaded page data: 2018/09/09, Race 1
All data wrote to files

Loaded page data: 2018/09/09, Race 10
All data wrote to files

Loaded page data: 2018/09/09, Race 2
All data wrote to files

Loaded page data: 2018/09/09, Race 3
All data wrote to files

Loaded page data: 2018/09/09, Race 4
All data wrote to files

Loaded page data: 2018/09/09, Race 5
All data wrote to files

Loaded page data: 2018/09/09, Race 6
All data wrote to files

Loaded page data: 2018/09/09, Race 7
All data wrote to files

Loaded page data: 2018/09/09, Race 8
All data wrote to files

Loaded page data: 2018/09/09, Race 9
All data wrote to files

Loaded page data: 2018/09/12, Race 1
All data wrote to files

Loaded page data: 2018/09/12, Race 2
All data wrote to files

Loaded 

Loaded page data: 2018/11/07, Race 4
All data wrote to files

Loaded page data: 2018/11/07, Race 5
All data wrote to files

Loaded page data: 2018/11/07, Race 6
All data wrote to files

Loaded page data: 2018/11/07, Race 7
All data wrote to files

Loaded page data: 2018/11/07, Race 8
All data wrote to files

Loaded page data: 2018/11/10, Race 1
All data wrote to files

Loaded page data: 2018/11/10, Race 10
All data wrote to files

Loaded page data: 2018/11/10, Race 2
All data wrote to files

Loaded page data: 2018/11/10, Race 3
All data wrote to files

Loaded page data: 2018/11/10, Race 4
All data wrote to files

Loaded page data: 2018/11/10, Race 5
All data wrote to files

Loaded page data: 2018/11/10, Race 6
All data wrote to files

Loaded page data: 2018/11/10, Race 7
All data wrote to files

Loaded page data: 2018/11/10, Race 8
All data wrote to files

Loaded page data: 2018/11/10, Race 9
All data wrote to files

Loaded page data: 2018/11/14, Race 1
All data wrote to files

Loaded 

Loaded page data: 2018/12/29, Race 1
All data wrote to files

Loaded page data: 2018/12/29, Race 10
All data wrote to files

Loaded page data: 2018/12/29, Race 2
All data wrote to files

Loaded page data: 2018/12/29, Race 3
All data wrote to files

Loaded page data: 2018/12/29, Race 4
All data wrote to files

Loaded page data: 2018/12/29, Race 5
All data wrote to files

Loaded page data: 2018/12/29, Race 6
All data wrote to files

Loaded page data: 2018/12/29, Race 7
All data wrote to files

Loaded page data: 2018/12/29, Race 8
All data wrote to files

Loaded page data: 2018/12/29, Race 9
All data wrote to files

Loaded page data: 2019/01/01, Race 1
All data wrote to files

Loaded page data: 2019/01/01, Race 10
All data wrote to files

Loaded page data: 2019/01/01, Race 11
All data wrote to files

Loaded page data: 2019/01/01, Race 2
All data wrote to files

Loaded page data: 2019/01/01, Race 3
All data wrote to files

Loaded page data: 2019/01/01, Race 4
All data wrote to files

Loade

...

Loaded page data: 2019/06/16, Race 4
All data wrote to files

Loaded page data: 2019/06/16, Race 5
All data wrote to files

Loaded page data: 2019/06/16, Race 6
All data wrote to files

Loaded page data: 2019/06/16, Race 7
All data wrote to files

Loaded page data: 2019/06/16, Race 8
All data wrote to files

Loaded page data: 2019/06/16, Race 9
All data wrote to files

Loaded page data: 2019/06/23, Race 1
All data wrote to files

Loaded page data: 2019/06/23, Race 10
All data wrote to files

Loaded page data: 2019/06/23, Race 2
All data wrote to files

Loaded page data: 2019/06/23, Race 3
All data wrote to files

Loaded page data: 2019/06/23, Race 4
All data wrote to files

Loaded page data: 2019/06/23, Race 5
All data wrote to files

Loaded page data: 2019/06/23, Race 6
All data wrote to files

Loaded page data: 2019/06/23, Race 7
All data wrote to files

Loaded page data: 2019/06/23, Race 8
All data wrote to files

Loaded page data: 2019/06/23, Race 9
All data wrote to files

Loaded 

Loaded page data: 2019/10/01, Race 2
All data wrote to files

Loaded page data: 2019/10/01, Race 3
All data wrote to files

Loaded page data: 2019/10/01, Race 4
All data wrote to files

Loaded page data: 2019/10/01, Race 5
All data wrote to files

Loaded page data: 2019/10/01, Race 6
All data wrote to files

Loaded page data: 2019/10/01, Race 7
All data wrote to files

Loaded page data: 2019/10/01, Race 8
All data wrote to files

Loaded page data: 2019/10/01, Race 9
All data wrote to files

Loaded page data: 2019/10/16, Race 1
All data wrote to files

Loaded page data: 2019/10/16, Race 2
All data wrote to files

Loaded page data: 2019/10/16, Race 3
All data wrote to files

Loaded page data: 2019/10/16, Race 4
All data wrote to files

Loaded page data: 2019/10/16, Race 5
All data wrote to files

Loaded page data: 2019/10/16, Race 6
All data wrote to files

Loaded page data: 2019/10/16, Race 7
All data wrote to files

Loaded page data: 2019/10/16, Race 8
All data wrote to files

Loaded p

Loaded page data: 2019/12/04, Race 5
All data wrote to files

Loaded page data: 2019/12/04, Race 6
All data wrote to files

Loaded page data: 2019/12/04, Race 7
All data wrote to files

Loaded page data: 2019/12/04, Race 8
All data wrote to files

Loaded page data: 2019/12/04, Race 9
All data wrote to files

Loaded page data: 2019/12/08, Race 1
All data wrote to files

Loaded page data: 2019/12/08, Race 10
All data wrote to files

Loaded page data: 2019/12/08, Race 2
All data wrote to files

Loaded page data: 2019/12/08, Race 3
All data wrote to files

Loaded page data: 2019/12/08, Race 4
All data wrote to files

Loaded page data: 2019/12/08, Race 5
All data wrote to files

Loaded page data: 2019/12/08, Race 6
All data wrote to files

Loaded page data: 2019/12/08, Race 7
All data wrote to files

Loaded page data: 2019/12/08, Race 8
All data wrote to files

Loaded page data: 2019/12/08, Race 9
All data wrote to files

Loaded page data: 2019/12/11, Race 1
All data wrote to files

Loaded 

Loaded page data: 2020/01/27, Race 2
All data wrote to files

Loaded page data: 2020/01/27, Race 3
All data wrote to files

Loaded page data: 2020/01/27, Race 4
All data wrote to files

Loaded page data: 2020/01/27, Race 5
All data wrote to files

Loaded page data: 2020/01/27, Race 6
All data wrote to files

Loaded page data: 2020/01/27, Race 7
All data wrote to files

Loaded page data: 2020/01/27, Race 8
All data wrote to files

Loaded page data: 2020/01/27, Race 9
All data wrote to files

Loaded page data: 2020/01/29, Race 1
All data wrote to files

Loaded page data: 2020/01/29, Race 2
All data wrote to files

Loaded page data: 2020/01/29, Race 3
All data wrote to files

Loaded page data: 2020/01/29, Race 4
All data wrote to files

Loaded page data: 2020/01/29, Race 5
All data wrote to files

Loaded page data: 2020/01/29, Race 6
All data wrote to files

Loaded page data: 2020/01/29, Race 7
All data wrote to files

Loaded page data: 2020/01/29, Race 8
All data wrote to files

Loaded p

Loaded page data: 2020/03/14, Race 6
All data wrote to files

Loaded page data: 2020/03/14, Race 7
All data wrote to files

Loaded page data: 2020/03/14, Race 8
All data wrote to files

Loaded page data: 2020/03/14, Race 9
All data wrote to files

Loaded page data: 2020/03/18, Race 1
All data wrote to files

Loaded page data: 2020/03/18, Race 2
All data wrote to files

Loaded page data: 2020/03/18, Race 3
All data wrote to files

Loaded page data: 2020/03/18, Race 4
All data wrote to files

Loaded page data: 2020/03/18, Race 5
All data wrote to files

Loaded page data: 2020/03/18, Race 6
All data wrote to files

Loaded page data: 2020/03/18, Race 7
All data wrote to files

Loaded page data: 2020/03/18, Race 8
All data wrote to files

Loaded page data: 2020/03/22, Race 1
All data wrote to files

Loaded page data: 2020/03/22, Race 10
All data wrote to files

Loaded page data: 2020/03/22, Race 2
All data wrote to files

Loaded page data: 2020/03/22, Race 3
All data wrote to files

Loaded 